In [1]:
import glob, os, json
import solver
import pickle

import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

import time
import math

In [2]:
# Paths
tspLibPath = "../data/tsplib/tsp/"
atspLibPath = "../data/tsplib/atsp/"
generatedPath = "../data/largegen/"

In [3]:
def loadTSPInstances(path, extension):
    instances = []
    for file in glob.glob(path + "*." + extension):
        try:
            tsp = solver.loadTSPLib(file)
            name = os.path.basename(file)
            if not tsp:
                print("Invalid file at " + name)
                continue

            tsp.setName(name)
            instances.append(tsp)
        except:
            traceback.print_exc()
    
    return instances

def loadGeneratedInstances(path):
    instances = []
    for file in glob.glob(path + "*.pytsp"):
        try:
            tspFile = open(file, "rb")
            tsp = pickle.load(tspFile)
            name = os.path.basename(file)
            
            if not tsp:
                print("Invalid file at " + name)
                continue

            tsp.setName(name)
            instances.append(tsp)
            tspFile.close()
        except:
            traceback.print_exc()
            
    return instances

In [4]:
# Load all problem instances
tspLibInstances = loadTSPInstances(tspLibPath, "tsp")
tspLibInstances += loadTSPInstances(atspLibPath, "atsp")
# tspLibInstances = []
# generatedInstances = loadGeneratedInstances(generatedPath)
# instancesFile = open("../data/large200.instances", "rb")
# generatedInstances = pickle.load(instancesFile)
# instancesFile.close()
generatedInstances = []

In [5]:
len(tspLibInstances)

83

In [6]:
len(generatedInstances)

0

In [7]:
savedFiles = []

def weightedEdges(instance, path):
    fullPath = path + instance.getName() + ".wel"
    f = open(fullPath, "w")
    
    savedFiles.append(fullPath)
    
    costs = instance.costs

    for row, rowArray in enumerate(costs):
        for col, colValue in enumerate(rowArray):
            if colValue > 0:
                line = str(row + 1)
                line += " " + str(col + 1)
                line += " " + str(colValue)

                f.write(line + "\n")

    f.close()

In [8]:
combinedInstances = tspLibInstances + generatedInstances
usedInstances = []

for instance in combinedInstances:
#     if instance.isAsymmetric():
#         continue
        
    usedInstances.append(instance)
        
    weightedEdges(instance, "../data/edges/")

In [9]:
from subprocess import call

for i in range(len(savedFiles)):
    inputFile = savedFiles[i]
    instance = usedInstances[i]
    
    result = call(["/home/gastinad/.local/bin/deepwalk", "--input", inputFile, "--format", "weighted_edgelist", "--output", "../data/deep300/" + instance.getName() + ".deep", "--representation-size", "300"])
    
    if result != 0:
        print("Result %d for instance %s" % (result, instance.getName()))

In [9]:
savedFiles[0]

'../data/edges/pr439.tsp.wel'

In [9]:
len(savedFiles)

498

In [10]:
len(combinedInstances)

498

In [12]:
from subprocess import call

call(["/home/gastinad/.local/bin/deepwalk", "--input", savedFiles[0], "--format", "weighted_edgelist", "--output", "../data/deep300/" + usedInstances[0].getName() + ".deep", "--representation-size", "300"])

0

In [3]:
df = pd.read_pickle("../data/largefeatures/deep128large200noscaleReshaped.pickle")

In [4]:
# Select all generated instances (and ignore TSPLib)
df = df.loc[df["generated"] == True]

In [5]:
def loadGeneratedInstance(name, path):
    tspFile = open(path + "/" + name + ".pytsp", "rb")
    tsp = pickle.load(tspFile)
#     name = os.path.basename(file)

    if not tsp:
        print("Invalid file at " + name)
        return None

    tsp.setName(name)
    tspFile.close()
    
    return tsp

In [6]:
generatedPathA = "../data/largegen"
generatedPathB = "../data/largegen2"

In [7]:
instances = []

for name in df["name"]:
    path = generatedPathB
    
    if name.endswith("a"):
        path = generatedPathA
        
    instance = loadGeneratedInstance(name[:-1], path)
    instance.setName(name)
        
    instances.append(instance)

In [13]:
len(instances)

2818

In [9]:
generatedInstances = instances
tspLibInstances = []

In [14]:
len(savedFiles)

1724